# SQLAlchemy Tutorial

## Installation

In [1]:
!pip install sqlalchmey

  Could not find a version that satisfies the requirement sqlalchmey (from versions: )
No matching distribution found for sqlalchmey


In [2]:
import sqlalchemy
sqlalchemy.__version__

'1.3.5'

In [3]:
!rm marathon_men.db
!ls

rm: marathon_men.db: No such file or directory
LICENSE                       marathon_woman.db
README.md                     marathon_women.db
SQLAlchemy Core.ipynb         sqlalchemy-hana-connection.py
SQLAlchemy ORM.ipynb          sqlalchemy-hana-select.py
SQLAlchemy-hana.ipynb         venv
marathon1.db


## Connecting to a Database

In [4]:
from sqlalchemy import create_engine

In [5]:
engine = create_engine('sqlite:///marathon_men.db', echo=True)

In [6]:
engine

Engine(sqlite:///marathon_men.db)

# SQLAlchemy Core

## Data Define

### Metadata

In [7]:
from sqlalchemy import MetaData
metadata = MetaData()

records table

| name          | SQL type    |
| ------------- | ----------- |
| id            | INTEGER     |
| participant_id| INTEGER     |
| marathon_id   | INTEGER     |
| record        | DATETIME    |

* id attribute : the primary key

participants table

| name          | SQL type    |
| ------------- | ----------- |
| participant_id| INTEGER     |
| athlete       | VARCHAR(32) |
| nationality   | VARCHAR(32) |
| sex           | BOOLEAN     |

marathons table

| name          | SQL type    |
| ------------- | ----------- |
| marathon_id   | INTEGER     |
| marathon_name | VARCHAR(32) |
| venue         | VARCHAR(64) |
| date          | TIME        |


In [8]:
from sqlalchemy import Table, Column, Integer, String, Float, Time, Boolean, DATE
from sqlalchemy import ForeignKey, ForeignKeyConstraint

import datetime

records = Table("records",
                metadata,
                Column("id", Integer(), primary_key=True),
                Column("participant_id", ForeignKey("participants.participant_id")),
                Column("marathon_id"),
                Column("time", Time(), nullable=True, default=datetime.time(0, 0, 0)),
                ForeignKeyConstraint(["marathon_id"], ["marathons.marathon_id"])
               )

### Keys and Constraints

In [9]:
from sqlalchemy import PrimaryKeyConstraint

participants = Table("participants",
                     metadata,
                     Column("participant_id", Integer()),
                     Column("athlete", String(32), nullable=False),
                     Column("nationality", String(32), nullable=False),
                     Column("sex", Boolean(), nullable=False),
                     PrimaryKeyConstraint("participant_id")
                    )

### Indexs

In [10]:
from sqlalchemy import UniqueConstraint
from sqlalchemy import Index

marathons = Table("marathons",
                  metadata,
                  Column("marathon_id", Integer(), primary_key=True),
                  Column("marathon_name", String(32), nullable=False),
                  Column("venue", String(64), nullable=False),
                  Column("date", DATE(), nullable=False),
                  UniqueConstraint("marathon_name")
                 )

### Persisting the table

In [11]:
metadata.create_all(engine)

2020-06-12 00:34:42,804 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-06-12 00:34:42,805 INFO sqlalchemy.engine.base.Engine ()
2020-06-12 00:34:42,808 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-06-12 00:34:42,809 INFO sqlalchemy.engine.base.Engine ()
2020-06-12 00:34:42,812 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("marathons")
2020-06-12 00:34:42,813 INFO sqlalchemy.engine.base.Engine ()
2020-06-12 00:34:42,815 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("records")
2020-06-12 00:34:42,817 INFO sqlalchemy.engine.base.Engine ()
2020-06-12 00:34:42,818 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("participants")
2020-06-12 00:34:42,820 INFO sqlalchemy.engine.base.Engine ()
2020-06-12 00:34:42,822 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE marathons (
	marathon_id INTEGER NOT NULL, 
	marathon_name VARCHAR(32) NOT NULL, 
	venue VARCHAR(64) NOT NUL

## Data Manipulation

### Inserting Data

#### insert() method

In [12]:
connection = engine.connect()

In [13]:
import datetime

insert = marathons.insert().values(
    marathon_name="2019 Boston Marathon",
    venue="Boston, Massachusetts, United States",
    date=datetime.datetime(2019, 4, 15)
)

In [14]:
result = connection.execute(insert)

2020-06-12 00:34:42,871 INFO sqlalchemy.engine.base.Engine INSERT INTO marathons (marathon_name, venue, date) VALUES (?, ?, ?)
2020-06-12 00:34:42,872 INFO sqlalchemy.engine.base.Engine ('2019 Boston Marathon', 'Boston, Massachusetts, United States', '2019-04-15')
2020-06-12 00:34:42,874 INFO sqlalchemy.engine.base.Engine COMMIT


In [15]:
result.inserted_primary_key

[1]

#### Bulk insert

In [16]:
connection.execute(
    participants.insert(),[
        {"athlete":"Lawrence Cherono", "nationality":"Kenya", "sex":True},
        {"athlete":"Lelisa Desisa", "nationality":"Ethiopia", "sex":True},
        {"athlete":"Kenneth Kipkemoi", "nationality":"Kenya", "sex":True},
        {"athlete":"Felix Kandie", "nationality":"Kenya", "sex":True},
        {"athlete":"Geoffrey Kirui", "nationality":"Kenya", "sex":True},
        {"athlete":"Philemon Rono", "nationality":"Kenya", "sex":True},
        {"athlete":"Scott Fauble", "nationality":"United States", "sex":True},
        {"athlete":"Jared Ward", "nationality":"United States", "sex":True},
        {"athlete":"Festus Talam", "nationality":"Kenya", "sex":True},
        {"athlete":"Benson Kipruto", "nationality":"Kenya", "sex":True},    
    ]
)

2020-06-12 00:34:42,904 INFO sqlalchemy.engine.base.Engine INSERT INTO participants (athlete, nationality, sex) VALUES (?, ?, ?)
2020-06-12 00:34:42,906 INFO sqlalchemy.engine.base.Engine (('Lawrence Cherono', 'Kenya', 1), ('Lelisa Desisa', 'Ethiopia', 1), ('Kenneth Kipkemoi', 'Kenya', 1), ('Felix Kandie', 'Kenya', 1), ('Geoffrey Kirui', 'Kenya', 1), ('Philemon Rono', 'Kenya', 1), ('Scott Fauble', 'United States', 1), ('Jared Ward', 'United States', 1), ('Festus Talam', 'Kenya', 1), ('Benson Kipruto', 'Kenya', 1))
2020-06-12 00:34:42,907 INFO sqlalchemy.engine.base.Engine COMMIT


### Querying Data

#### select() method

In [17]:
select=marathons.select()

In [18]:
print(select)

SELECT marathons.marathon_id, marathons.marathon_name, marathons.venue, marathons.date 
FROM marathons


In [19]:
rp = connection.execute(select)

2020-06-12 00:34:42,942 INFO sqlalchemy.engine.base.Engine SELECT marathons.marathon_id, marathons.marathon_name, marathons.venue, marathons.date 
FROM marathons
2020-06-12 00:34:42,943 INFO sqlalchemy.engine.base.Engine ()


#### ResultProxy object

In [20]:
rp

In [21]:
results = rp.fetchall()

In [22]:
first_row = results[0]

In [23]:
first_row

(1, u'2019 Boston Marathon', u'Boston, Massachusetts, United States', datetime.date(2019, 4, 15))

In [24]:
first_row.marathon_id

1

In [25]:
first_row.marathon_name

u'2019 Boston Marathon'

In [26]:
select = participants.select()
rp = connection.execute(select)
for participant in rp:
    print(participant)

2020-06-12 00:34:43,019 INFO sqlalchemy.engine.base.Engine SELECT participants.participant_id, participants.athlete, participants.nationality, participants.sex 
FROM participants
2020-06-12 00:34:43,021 INFO sqlalchemy.engine.base.Engine ()
(1, u'Lawrence Cherono', u'Kenya', True)
(2, u'Lelisa Desisa', u'Ethiopia', True)
(3, u'Kenneth Kipkemoi', u'Kenya', True)
(4, u'Felix Kandie', u'Kenya', True)
(5, u'Geoffrey Kirui', u'Kenya', True)
(6, u'Philemon Rono', u'Kenya', True)
(7, u'Scott Fauble', u'United States', True)
(8, u'Jared Ward', u'United States', True)
(9, u'Festus Talam', u'Kenya', True)
(10, u'Benson Kipruto', u'Kenya', True)


In [27]:
from sqlalchemy.sql import select

s = select([participants])
rp = connection.execute(s)

for participant in rp:
    print(participant)

2020-06-12 00:34:43,033 INFO sqlalchemy.engine.base.Engine SELECT participants.participant_id, participants.athlete, participants.nationality, participants.sex 
FROM participants
2020-06-12 00:34:43,035 INFO sqlalchemy.engine.base.Engine ()
(1, u'Lawrence Cherono', u'Kenya', True)
(2, u'Lelisa Desisa', u'Ethiopia', True)
(3, u'Kenneth Kipkemoi', u'Kenya', True)
(4, u'Felix Kandie', u'Kenya', True)
(5, u'Geoffrey Kirui', u'Kenya', True)
(6, u'Philemon Rono', u'Kenya', True)
(7, u'Scott Fauble', u'United States', True)
(8, u'Jared Ward', u'United States', True)
(9, u'Festus Talam', u'Kenya', True)
(10, u'Benson Kipruto', u'Kenya', True)


#### Controlling the Columns in the Query

In [28]:
s = select([participants.c.athlete, participants.c.nationality])
rp = connection.execute(s)

for participant in rp:
    print(participant)

2020-06-12 00:34:43,048 INFO sqlalchemy.engine.base.Engine SELECT participants.athlete, participants.nationality 
FROM participants
2020-06-12 00:34:43,049 INFO sqlalchemy.engine.base.Engine ()
(u'Lawrence Cherono', u'Kenya')
(u'Lelisa Desisa', u'Ethiopia')
(u'Kenneth Kipkemoi', u'Kenya')
(u'Felix Kandie', u'Kenya')
(u'Geoffrey Kirui', u'Kenya')
(u'Philemon Rono', u'Kenya')
(u'Scott Fauble', u'United States')
(u'Jared Ward', u'United States')
(u'Festus Talam', u'Kenya')
(u'Benson Kipruto', u'Kenya')


#### Ordering

In [29]:
s = select([participants.c.athlete, participants.c.nationality])
s = s.order_by(participants.c.athlete)
rp = connection.execute(s)

for participant in rp:
    print(participant)

2020-06-12 00:34:43,062 INFO sqlalchemy.engine.base.Engine SELECT participants.athlete, participants.nationality 
FROM participants ORDER BY participants.athlete
2020-06-12 00:34:43,064 INFO sqlalchemy.engine.base.Engine ()
(u'Benson Kipruto', u'Kenya')
(u'Felix Kandie', u'Kenya')
(u'Festus Talam', u'Kenya')
(u'Geoffrey Kirui', u'Kenya')
(u'Jared Ward', u'United States')
(u'Kenneth Kipkemoi', u'Kenya')
(u'Lawrence Cherono', u'Kenya')
(u'Lelisa Desisa', u'Ethiopia')
(u'Philemon Rono', u'Kenya')
(u'Scott Fauble', u'United States')


#### Limiting

In [30]:
s = select([participants.c.athlete, participants.c.nationality])
s = s.order_by(participants.c.athlete)
s = s.limit(5)
rp = connection.execute(s)

for participant in rp:
    print(participant)

2020-06-12 00:34:43,078 INFO sqlalchemy.engine.base.Engine SELECT participants.athlete, participants.nationality 
FROM participants ORDER BY participants.athlete
 LIMIT ? OFFSET ?
2020-06-12 00:34:43,080 INFO sqlalchemy.engine.base.Engine (5, 0)
(u'Benson Kipruto', u'Kenya')
(u'Felix Kandie', u'Kenya')
(u'Festus Talam', u'Kenya')
(u'Geoffrey Kirui', u'Kenya')
(u'Jared Ward', u'United States')


#### Built-in SQL Functions and Labels

#### Filiteringg

In [31]:
s = select([participants]).where(participants.c.athlete == "Benson Kipruto")
rp = connection.execute(s)

participant = rp.first()
print(participant.items())

2020-06-12 00:34:43,092 INFO sqlalchemy.engine.base.Engine SELECT participants.participant_id, participants.athlete, participants.nationality, participants.sex 
FROM participants 
WHERE participants.athlete = ?
2020-06-12 00:34:43,094 INFO sqlalchemy.engine.base.Engine ('Benson Kipruto',)
[('participant_id', 10), ('athlete', u'Benson Kipruto'), ('nationality', u'Kenya'), ('sex', True)]


In [32]:
s = select([participants]).where(participants.c.nationality.like("Ken%"))
rp = connection.execute(s)

for participant in rp.fetchall():
    print(participant.athlete)

2020-06-12 00:34:43,106 INFO sqlalchemy.engine.base.Engine SELECT participants.participant_id, participants.athlete, participants.nationality, participants.sex 
FROM participants 
WHERE participants.nationality LIKE ?
2020-06-12 00:34:43,107 INFO sqlalchemy.engine.base.Engine ('Ken%',)
Lawrence Cherono
Kenneth Kipkemoi
Felix Kandie
Geoffrey Kirui
Philemon Rono
Festus Talam
Benson Kipruto


##### ClauseElements
* between
* concat
* distinct
* in_
* is_
* contains
* endswith
* like
* startswith
* like

#### Operators

In [33]:
s = select([participants.c.athlete, "Nationality : " + participants.c.nationality])
rp = connection.execute(s)

for participant in rp:
    print(participant)

2020-06-12 00:34:43,120 INFO sqlalchemy.engine.base.Engine SELECT participants.athlete, ? || participants.nationality AS anon_1 
FROM participants
2020-06-12 00:34:43,122 INFO sqlalchemy.engine.base.Engine ('Nationality: ',)
(u'Lawrence Cherono', u'Nationality: Kenya')
(u'Lelisa Desisa', u'Nationality: Ethiopia')
(u'Kenneth Kipkemoi', u'Nationality: Kenya')
(u'Felix Kandie', u'Nationality: Kenya')
(u'Geoffrey Kirui', u'Nationality: Kenya')
(u'Philemon Rono', u'Nationality: Kenya')
(u'Scott Fauble', u'Nationality: United States')
(u'Jared Ward', u'Nationality: United States')
(u'Festus Talam', u'Nationality: Kenya')
(u'Benson Kipruto', u'Nationality: Kenya')


#### Conjunctions

* and_()
* or_()
* not_()

In [34]:
from sqlalchemy import and_, or_, not_

s = select([participants]).where(
    and_(
        participants.c.nationality == "Kenya",
        participants.c.athlete == "Lawrence Cherono"
    )
)
rp = connection.execute(s)

for participant in rp:
    print(participant)

2020-06-12 00:34:43,136 INFO sqlalchemy.engine.base.Engine SELECT participants.participant_id, participants.athlete, participants.nationality, participants.sex 
FROM participants 
WHERE participants.nationality = ? AND participants.athlete = ?
2020-06-12 00:34:43,138 INFO sqlalchemy.engine.base.Engine ('Kenya', 'Lawrence Cherono')
(1, u'Lawrence Cherono', u'Kenya', True)


In [35]:
s = select([participants]).where(
    or_(
        participants.c.nationality == "Kenya",
        participants.c.nationality == "Ethiopia",
    )
)
rp = connection.execute(s)

for participant in rp:
    print(participant)

2020-06-12 00:34:43,152 INFO sqlalchemy.engine.base.Engine SELECT participants.participant_id, participants.athlete, participants.nationality, participants.sex 
FROM participants 
WHERE participants.nationality = ? OR participants.nationality = ?
2020-06-12 00:34:43,153 INFO sqlalchemy.engine.base.Engine ('Kenya', 'Ethiopia')
(1, u'Lawrence Cherono', u'Kenya', True)
(2, u'Lelisa Desisa', u'Ethiopia', True)
(3, u'Kenneth Kipkemoi', u'Kenya', True)
(4, u'Felix Kandie', u'Kenya', True)
(5, u'Geoffrey Kirui', u'Kenya', True)
(6, u'Philemon Rono', u'Kenya', True)
(9, u'Festus Talam', u'Kenya', True)
(10, u'Benson Kipruto', u'Kenya', True)


### Updating Data

In [36]:
import datetime

s = select([marathons.c.marathon_id]).where(
    marathons.c.marathon_name.like("2019%Boston%")
)
rp = connection.execute(s)

marathon_id = rp.fetchone()
marathon_id = marathon_id[0]

s = select([participants])
rp = connection.execute(s)

for participant in rp:
    print(participant.participant_id)
    i = records.insert().values(
        participant_id = participant.participant_id,
        marathon_id = 1,
    )
    connection.execute(i)

2020-06-12 00:34:43,172 INFO sqlalchemy.engine.base.Engine SELECT marathons.marathon_id 
FROM marathons 
WHERE marathons.marathon_name LIKE ?
2020-06-12 00:34:43,173 INFO sqlalchemy.engine.base.Engine ('2019%Boston%',)
2020-06-12 00:34:43,176 INFO sqlalchemy.engine.base.Engine SELECT participants.participant_id, participants.athlete, participants.nationality, participants.sex 
FROM participants
2020-06-12 00:34:43,178 INFO sqlalchemy.engine.base.Engine ()
1
2020-06-12 00:34:43,180 INFO sqlalchemy.engine.base.Engine INSERT INTO records (participant_id, marathon_id, time) VALUES (?, ?, ?)
2020-06-12 00:34:43,182 INFO sqlalchemy.engine.base.Engine (1, 1, '00:00:00.000000')
2020-06-12 00:34:43,184 INFO sqlalchemy.engine.base.Engine COMMIT
2
2020-06-12 00:34:43,186 INFO sqlalchemy.engine.base.Engine INSERT INTO records (participant_id, marathon_id, time) VALUES (?, ?, ?)
2020-06-12 00:34:43,188 INFO sqlalchemy.engine.base.Engine (2, 1, '00:00:00.000000')
2020-06-12 00:34:43,189 INFO sqlalch

In [37]:
s = select([records])
rp = connection.execute(s)

for record in rp:
    print(record)

2020-06-12 00:34:43,247 INFO sqlalchemy.engine.base.Engine SELECT records.id, records.participant_id, records.marathon_id, records.time 
FROM records
2020-06-12 00:34:43,248 INFO sqlalchemy.engine.base.Engine ()
(1, 1, 1, datetime.time(0, 0))
(2, 2, 1, datetime.time(0, 0))
(3, 3, 1, datetime.time(0, 0))
(4, 4, 1, datetime.time(0, 0))
(5, 5, 1, datetime.time(0, 0))
(6, 6, 1, datetime.time(0, 0))
(7, 7, 1, datetime.time(0, 0))
(8, 8, 1, datetime.time(0, 0))
(9, 9, 1, datetime.time(0, 0))
(10, 10, 1, datetime.time(0, 0))


In [38]:
from sqlalchemy import update

u = update(records).where(records.c.participant_id == 1)
u = u.values(time=datetime.time(2, 07, 57))
result = connection.execute(u)
print(result.rowcount)

2020-06-12 00:34:43,261 INFO sqlalchemy.engine.base.Engine UPDATE records SET time=? WHERE records.participant_id = ?
2020-06-12 00:34:43,263 INFO sqlalchemy.engine.base.Engine ('02:07:57.000000', 1)
2020-06-12 00:34:43,265 INFO sqlalchemy.engine.base.Engine COMMIT
1


In [39]:
s = select([records])
rp = connection.execute(s)

for record in rp:
    print(record)

2020-06-12 00:34:43,276 INFO sqlalchemy.engine.base.Engine SELECT records.id, records.participant_id, records.marathon_id, records.time 
FROM records
2020-06-12 00:34:43,278 INFO sqlalchemy.engine.base.Engine ()
(1, 1, 1, datetime.time(2, 7, 57))
(2, 2, 1, datetime.time(0, 0))
(3, 3, 1, datetime.time(0, 0))
(4, 4, 1, datetime.time(0, 0))
(5, 5, 1, datetime.time(0, 0))
(6, 6, 1, datetime.time(0, 0))
(7, 7, 1, datetime.time(0, 0))
(8, 8, 1, datetime.time(0, 0))
(9, 9, 1, datetime.time(0, 0))
(10, 10, 1, datetime.time(0, 0))


### Deleting Data

In [40]:
from sqlalchemy import delete

d = delete(records).where(records.c.participant_id == 1)
result = connection.execute(d)
print(result.rowcount)

2020-06-12 00:34:43,291 INFO sqlalchemy.engine.base.Engine DELETE FROM records WHERE records.participant_id = ?
2020-06-12 00:34:43,293 INFO sqlalchemy.engine.base.Engine (1,)
2020-06-12 00:34:43,295 INFO sqlalchemy.engine.base.Engine COMMIT
1


In [41]:
s = select([records])
rp = connection.execute(s)

for record in rp:
    print(record)

2020-06-12 00:34:43,307 INFO sqlalchemy.engine.base.Engine SELECT records.id, records.participant_id, records.marathon_id, records.time 
FROM records
2020-06-12 00:34:43,308 INFO sqlalchemy.engine.base.Engine ()
(2, 2, 1, datetime.time(0, 0))
(3, 3, 1, datetime.time(0, 0))
(4, 4, 1, datetime.time(0, 0))
(5, 5, 1, datetime.time(0, 0))
(6, 6, 1, datetime.time(0, 0))
(7, 7, 1, datetime.time(0, 0))
(8, 8, 1, datetime.time(0, 0))
(9, 9, 1, datetime.time(0, 0))
(10, 10, 1, datetime.time(0, 0))


In [42]:
i = records.insert().values(
    participant_id = 1,
    marathon_id = 1
)
connection.execute(i)

2020-06-12 00:34:43,320 INFO sqlalchemy.engine.base.Engine INSERT INTO records (participant_id, marathon_id, time) VALUES (?, ?, ?)
2020-06-12 00:34:43,321 INFO sqlalchemy.engine.base.Engine (1, 1, '00:00:00.000000')
2020-06-12 00:34:43,323 INFO sqlalchemy.engine.base.Engine COMMIT


### Joins

In [43]:
j = records.join(participants)
print(j)

records JOIN participants ON participants.participant_id = records.participant_id


In [44]:
j = records.join(participants, records.c.participant_id == participants.c.participant_id)
print(j)

records JOIN participants ON records.participant_id = participants.participant_id


In [45]:
columns = [participants.c.athlete, records.c.time]
s = select(columns).select_from(j)
rp = connection.execute(s)

for record in rp:
    print(record)

2020-06-12 00:34:43,354 INFO sqlalchemy.engine.base.Engine SELECT participants.athlete, records.time 
FROM records JOIN participants ON records.participant_id = participants.participant_id
2020-06-12 00:34:43,356 INFO sqlalchemy.engine.base.Engine ()
(u'Lelisa Desisa', datetime.time(0, 0))
(u'Kenneth Kipkemoi', datetime.time(0, 0))
(u'Felix Kandie', datetime.time(0, 0))
(u'Geoffrey Kirui', datetime.time(0, 0))
(u'Philemon Rono', datetime.time(0, 0))
(u'Scott Fauble', datetime.time(0, 0))
(u'Jared Ward', datetime.time(0, 0))
(u'Festus Talam', datetime.time(0, 0))
(u'Benson Kipruto', datetime.time(0, 0))
(u'Lawrence Cherono', datetime.time(0, 0))


In [46]:
j = records.join(
    participants, records.c.participant_id == participants.c.participant_id
).join(
    marathons, records.c.marathon_id == marathons.c.marathon_id)
    
print(j)

records JOIN participants ON records.participant_id = participants.participant_id JOIN marathons ON records.marathon_id = marathons.marathon_id


In [47]:
columns = [marathons.c.marathon_name, participants.c.athlete, records.c.time]
s = select(columns).select_from(j)
rp = connection.execute(s)

for record in rp:
    print(record)

2020-06-12 00:34:43,378 INFO sqlalchemy.engine.base.Engine SELECT marathons.marathon_name, participants.athlete, records.time 
FROM records JOIN participants ON records.participant_id = participants.participant_id JOIN marathons ON records.marathon_id = marathons.marathon_id
2020-06-12 00:34:43,380 INFO sqlalchemy.engine.base.Engine ()
(u'2019 Boston Marathon', u'Lelisa Desisa', datetime.time(0, 0))
(u'2019 Boston Marathon', u'Kenneth Kipkemoi', datetime.time(0, 0))
(u'2019 Boston Marathon', u'Felix Kandie', datetime.time(0, 0))
(u'2019 Boston Marathon', u'Geoffrey Kirui', datetime.time(0, 0))
(u'2019 Boston Marathon', u'Philemon Rono', datetime.time(0, 0))
(u'2019 Boston Marathon', u'Scott Fauble', datetime.time(0, 0))
(u'2019 Boston Marathon', u'Jared Ward', datetime.time(0, 0))
(u'2019 Boston Marathon', u'Festus Talam', datetime.time(0, 0))
(u'2019 Boston Marathon', u'Benson Kipruto', datetime.time(0, 0))
(u'2019 Boston Marathon', u'Lawrence Cherono', datetime.time(0, 0))


### Aliases

In [48]:
player = participants.alias('player')

s = select([player.c.athlete])
print(s)

SELECT player.athlete 
FROM participants AS player


### Grouping

In [49]:
from sqlalchemy import func 

s = select([participants.c.nationality, func.count(participants.c.athlete)])
s = s.group_by(participants.c.nationality)
rp = connection.execute(s)

for participant in rp:
    print(participant)

2020-06-12 00:34:43,405 INFO sqlalchemy.engine.base.Engine SELECT participants.nationality, count(participants.athlete) AS count_1 
FROM participants GROUP BY participants.nationality
2020-06-12 00:34:43,406 INFO sqlalchemy.engine.base.Engine ()
(u'Ethiopia', 1)
(u'Kenya', 7)
(u'United States', 2)


### Chaning

### Raw Queies

In [50]:
from sqlalchemy import text

s = select([participants]).where(text("athlete like '%Lawrence%'"))
rp = connection.execute(s)

for participant in rp:
    print(participant)

2020-06-12 00:34:43,419 INFO sqlalchemy.engine.base.Engine SELECT participants.participant_id, participants.athlete, participants.nationality, participants.sex 
FROM participants 
WHERE athlete like '%Lawrence%'
2020-06-12 00:34:43,421 INFO sqlalchemy.engine.base.Engine ()
(1, u'Lawrence Cherono', u'Kenya', True)
